In [1]:
import random
import gym
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import time

/home/conuwoo/.local/lib/python3.8/site-packages/ale_py/roms/__init__.py:89: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  ROMS = resolve_roms()
2022-07-27 14:07:39.276344: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-07-27 14:07:39.276374: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/conuwoo/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or 

In [2]:
def cur_model(num_Conv2D,num_filters,num_kernel,num_lstm_units,num_action, num_frames):
  model = Sequential()
  model.add(Dense(num_action))  
  model.add(LSTM(num_lstm_units)) # LSTM
  model.add(Dense(num_action))  
  model.compile(loss='mean_squared_error', optimizer='Adam') # model compiles
  return model

In [3]:
def reinforcement(env, model, n_games=400, epochs=16, epsilon_decrease=0.05, backsight=8):
  epsilon = 1 # epsilon allows the model to explore new actions
  state_history = []
  value_history = []
  env.reset() # reset the enviornment state when starting.
  obs = env.observation_space.sample() # array with shape (x,) with values corresponding to each observation of the game. x is the number of observation.

  for game_number in range(n_games):
    states = []
    actions = []
    values = []
    memory = [obs] * backsight # after np.sum(obs,axis=2), shape changes to 2d so reshape it into 3d and divide by (255*3) for standarlization. Then multiply backsight.
    # before np.sum, it will be 3d but np.sum will sum every [] inside of the 3d array.
    frames = 0

    while True:
      if random.random() > epsilon: #first state, epsilon will always bigger but because epsilon decreases incrementally during the game, model will favor optimized policy model compare to random action.
        action = np.argmax(model.predict(np.array([memory], dtype='float16'))[0]) # LSTM outputs a single value, so choose the first index.
      else:
        action = env.action_space.sample() # this is random action and outputs discrete integer value i.e. 1
      
      obs, reward, done, _info = env.step(action) # returns observation_spcae, amount of reward, boolean value if episode has terminated, and info about state.
      env.render()  
      time.sleep(0.05)
      if frames % 1 == 0: # for every 32th frames
        print('Game', game_number, 'frame', frames) 
        actions.append(action) # action taken gets added to the array.
        states.append(memory) # obersvation states array gets added to the array/
        values.append([0] * env.action_space.n) # if action_space.n = 2, then it will be in a shape of [0,0]
        memory.pop(0) # removes the first index from memory array.
        memory.append(obs) # and replace with current observation_space. 

        for i, scores in enumerate(values):
          scores[actions[i]] += reward # for example, if there 2 action space, then actions[i] will consist of (0,0),(0,1),(1,0),(1,1). it will then add reward.
      
      frames += 1 # frames added after every episode.
      
      if done:
        print('Game', game_number, 'lasted', frames, 'frames')
        env.reset() # resets state.
        break
    
    state_history.extend(states) # observation states gets added to history.
    value_history.extend(values) # value get added to history. 
    model.fit(np.array(state_history, dtype='float16'), np.array(value_history)) # model.fit(x,y), x is state_history and y is value_history. Thus it's using state_history to predict possible values. Learning occurs here.
    epsilon = max(0, epsilon - epsilon_decrease) # eplison decreases until 0.

In [4]:
env = gym.make("CartPole-v0")
model = cur_model(4, 8, 4,8, env.action_space.n, 8)

/home/conuwoo/.local/lib/python3.8/site-packages/gym/envs/registration.py:592: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/home/conuwoo/.local/lib/python3.8/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/conuwoo/.local/lib/python3.8/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
2022-07-27 14:07:40.292689: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open sh

In [5]:
reinforcement(env, model)

/home/conuwoo/.local/lib/python3.8/site-packages/gym/core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Game 0 frame 0
Game 0 frame 1
Game 0 frame 2
Game 0 frame 3
Game 0 frame 4
Game 0 frame 5
Game 0 frame 6
Game 0 frame 7
Game 0 frame 8
Game 0 frame 9
Game 0 frame 10
Game 0 frame 11
Game 0 frame 12
Game 0 frame 13
Game 0 frame 14
Game 0 frame 15
Game 0 frame 16
Game 0 frame 17
Game 0 frame 18
Game 0 frame 19
Game 0 frame 20
Game 0 frame 21
Game 0 frame 22
Game 0 frame 23
Game 0 frame 24
Game 0 frame 25
Game 0 frame 26
Game 0 frame 27
Game 0 lasted 28 frames
1/1 [==============================] - 0s 782us/step - loss: 141.2617
Game 1 frame 0
Game 1 frame 1
Game 1 frame 2
Game 1 frame 3
Game 1 frame 4
Game 1 frame 5
Game 1 frame 6
Game 1 frame 7
Game 1 frame 8
Game 1 frame 9
Game 1 frame 10
Game 1 frame 11
Game 1 frame 12
Game 1 frame 13
Game 1 frame 14
Game 1 frame 15
Game 1 lasted 16 frames
2/2 [==============================] - 0s 3ms/step - loss: 105.0814
Game 2 frame 0
Game 2 frame 1
Game 2 frame 2
Game 2 frame 3
Game 2 frame 4
Game 2 frame 5
Game 2 frame 6
Game 2 frame 7
Game 2 fra

KeyboardInterrupt: 